In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE40912"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE40912"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE40912.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE40912.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE40912.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations (RCC survival)"
!Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarrays with 44k-probes were used to interrogate 17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes) split into four pools. Meta-analyses were performed by taking the genomic coordinates of the RCC-expressed lncRNAs, and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Is gene expression data available?
is_gene_available = True  # Based on the background info, this dataset contains gene expression data (microarray)

# Step 2: Clinical feature extraction

# 2.1 Identify the row indices for trait, age, and gender
trait_row = 14  # 'patient status' gives survival information related to Kidney cancer
age_row = 4     # 'age at surgery (yrs)' gives age information
gender_row = 3  # 'gender' gives gender information

# 2.2 Define conversion functions for each variable

def convert_trait(value):
    """
    Convert trait value to binary (1 for death from cancer, 0 for alive without cancer)
    """
    if not isinstance(value, str) or ":" not in value:
        return None
    status = value.split(":", 1)[1].strip().lower()
    if "death" in status:
        return 1
    elif "alive" in status:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age value to continuous numeric value
    """
    if not isinstance(value, str) or ":" not in value:
        return None
    try:
        age_str = value.split(":", 1)[1].strip()
        return float(age_str)
    except:
        return None

def convert_gender(value):
    """
    Convert gender value to binary (1 for male, 0 for female)
    """
    if not isinstance(value, str) or ":" not in value:
        return None
    gender = value.split(":", 1)[1].strip().lower()
    if "male" == gender:
        return 1
    elif "female" == gender:
        return 0
    else:
        return None

# Step 3: Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical feature extraction (only if trait_row is not None)
if trait_row is not None:
    # Sample characteristics data from previous step
    sample_chars = {0: ['patient identifier: 1', 'patient identifier: 3', 'patient identifier: 5', 'patient identifier: 7', 'patient identifier: 9', 'patient identifier: 10', 'patient identifier: 11', 'patient identifier: 13', 'patient identifier: 15', 'patient identifier: 24', 'patient identifier: 26', 'patient identifier: 28', 'patient identifier: 29', 'patient identifier: 30', 'patient identifier: 32', 'patient identifier: 33'], 
                   1: ['disease: clear cell renal cell carcinoma (RCC)'], 
                   2: ['tissue: kidney tumor'], 
                   3: ['gender: male', 'gender: female'], 
                   4: ['age at surgery (yrs): 51', 'age at surgery (yrs): 78', 'age at surgery (yrs): 53', 'age at surgery (yrs): 41', 'age at surgery (yrs): 39', 'age at surgery (yrs): 34', 'age at surgery (yrs): 66', 'age at surgery (yrs): 75', 'age at surgery (yrs): 40', 'age at surgery (yrs): 63', 'age at surgery (yrs): 35'], 
                   5: ['fuhrman grade: III', 'fuhrman grade: IV', 'fuhrman grade: II'], 
                   6: ['tumor size (cm): 18', 'tumor size (cm): 6', 'tumor size (cm): 8', 'tumor size (cm): 11', 'tumor size (cm): 6.5', 'tumor size (cm): 7', 'tumor size (cm): 5', 'tumor size (cm): 10', 'tumor size (cm): 15', 'tumor size (cm): 20', 'tumor size (cm): 8.5', 'tumor size (cm): 13', 'tumor size (cm): 4'], 
                   7: ['necrosis: yes', 'necrosis: no'], 
                   8: ['capsule infiltration: yes', 'capsule infiltration: no'], 
                   9: ['tnm classification (t): 3c', 'tnm classification (t): 2', 'tnm classification (t): 3a', 'tnm classification (t): 1b', 'tnm classification (t): 3', 'tnm classification (t): 3b', 'tnm classification (t): 1', 'tnm classification (t): 1a'], 
                   10: ['tnm classification (n): no data available', 'tnm classification (n): 1', 'tnm classification (n): 0', 'tnm classification (n): 2'], 
                   11: ['tnm classification (m): no data available', 'tnm classification (m): 1'], 
                   12: ['organ metastasis at surgery: no data available', 'organ metastasis at surgery: endocava, bones', 'organ metastasis at surgery: liver', 'organ metastasis at surgery: lung', 'organ metastasis at surgery: peritoneum'], 
                   13: ['organ metastasis after surgery: no data available', 'organ metastasis after surgery: liver, spleen', 'organ metastasis after surgery: bones', 'organ metastasis after surgery: brain, lung, bones'], 
                   14: ['patient status: cancer-specific death', 'patient status: alive without cancer'], 
                   15: ['follow-up (months): 0', 'follow-up (months): 21', 'follow-up (months): 6', 'follow-up (months): 66', 'follow-up (months): 60', 'follow-up (months): 8', 'follow-up (months): 16', 'follow-up (months): 62', 'follow-up (months): 54', 'follow-up (months): 56', 'follow-up (months): 17']}
    
    # Create a properly structured DataFrame from the sample characteristics
    # We need to create a DataFrame where each row is a feature and each column is a sample
    max_samples = max(len(values) for values in sample_chars.values())
    sample_ids = [f'GSM{1000+i}' for i in range(max_samples)]
    
    # Initialize a DataFrame with the rows being the feature types
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=sample_ids)
    
    # Fill in the data
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            if col_idx < max_samples:
                clinical_data.iloc[row_idx, col_idx] = value
    
    # Extract clinical features using the library function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    clinical_preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(clinical_preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Saved clinical features to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM1000': [1.0, 51.0, 1.0], 'GSM1001': [0.0, 78.0, 0.0], 'GSM1002': [nan, 53.0, nan], 'GSM1003': [nan, 41.0, nan], 'GSM1004': [nan, 39.0, nan], 'GSM1005': [nan, 34.0, nan], 'GSM1006': [nan, 66.0, nan], 'GSM1007': [nan, 75.0, nan], 'GSM1008': [nan, 40.0, nan], 'GSM1009': [nan, 63.0, nan], 'GSM1010': [nan, 35.0, nan], 'GSM1011': [nan, nan, nan], 'GSM1012': [nan, nan, nan], 'GSM1013': [nan, nan, nan], 'GSM1014': [nan, nan, nan], 'GSM1015': [nan, nan, nan]}
Saved clinical features to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE40912.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 3205 rows
First 20 gene IDs:
Index(['1', '2', '3', '4', '5', '6', '7', '9', '10', '11', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The identifiers seen in the gene data are numeric IDs (1, 2, 3, etc.) which are not human gene symbols.
# These appear to be Entrez Gene IDs or possibly probe IDs that need to be mapped to gene symbols.
# As a domain expert, I can confirm these numeric identifiers require mapping to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...
Successfully extracted gene annotation data with 107200 rows

Gene annotation preview (first few rows):
{'ID': ['910', '4260', '1981', '2381', '4288'], 'GB_ACC': ['BE833259', 'BE702227', 'BF364095', 'BE081005', 'AW880607'], 'SPOT_TYPE': ['Exonic', 'Exonic', 'Exonic', 'Exonic', 'Exonic'], 'GENE_ID': [85439.0, 2776.0, 84131.0, 2776.0, 54768.0], 'GENE_SYMBOL': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN'], 'GENE_ANNOTATION': ['stonin 2', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'centrosomal protein 78kDa', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'hydrocephalus inducing homolog 2 (mouse); hydrocephalus inducing homolog (mouse)'], 'CPC_CODING_POTENTIAL': ['noncoding', 'noncoding', 'noncoding', 'noncoding', '-'], 'SEQUENCE': ['CTGATCCGCTTAAGCTTAGTATGTTTGAGTGTGTAATTTTAGTTTCTTTTCTGGTTGTATTTGTGGTAGTCAGATGTGTTGGATTGATTCCAACTGGACAGAGTAAGGAATTCCAGCATCCTCTTCCTGCTTGCTCGTGTTACCCCACAGATCAAACCCTCAATTCTAGTTG

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns that contain gene identifiers and gene symbols in the annotation data
print("Mapping gene identifiers to gene symbols...")

# Based on the preview, the 'ID' column in annotation contains the same numeric identifiers
# that we saw in the gene expression data, and 'GENE_SYMBOL' contains the gene symbols
id_column = 'ID'
gene_symbol_column = 'GENE_SYMBOL'

# 2. Get a gene mapping dataframe using the helper function
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("Mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
print("\nConverting probe measurements to gene expression data...")
try:
    gene_data = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Successfully mapped to {len(gene_data.index)} genes")
    print("First 10 mapped gene symbols:")
    print(gene_data.index[:10].tolist())
    
    # Save the gene expression data to CSV
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Saved gene expression data to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error applying gene mapping: {e}")
    is_gene_available = False


Mapping gene identifiers to gene symbols...


Created gene mapping with 3169 entries
Mapping preview (first 5 rows):
     ID   Gene
0   910  STON2
1  4260   GNAQ
2  1981  CEP78
3  2381   GNAQ
4  4288  HYDIN

Converting probe measurements to gene expression data...


Successfully mapped to 2608 genes
First 10 mapped gene symbols:
['A2ML1', 'AARSD1', 'AATF', 'ABCA2', 'ABCA3', 'ABCB1', 'ABCB10', 'ABCB6', 'ABCB7', 'ABCC10']


Saved gene expression data to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE40912.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Load the clinical data that was previously saved in Step 2
print("\nLoading clinical data...")
try:
    # Check if the clinical data file exists
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        # Make sure the trait column is present
        if trait not in clinical_df.columns:
            # If we don't have the trait column, rename the first column (which should be the trait)
            clinical_df = clinical_df.rename(columns={clinical_df.columns[0]: trait})
        is_trait_available = True
    else:
        # If we don't have the file, use the clinical data from Step 2
        print("Clinical data file not found. Using the clinical features extracted in Step 2.")
        # We know from Step 2 that trait_row=14, age_row=4, gender_row=3
        clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=14,  # 'patient status' (survival info)
            convert_trait=convert_trait,
            age_row=4,     # 'age at surgery'
            convert_age=convert_age,
            gender_row=3,  # 'gender'
            convert_gender=convert_gender
        )
        is_trait_available = True
        
    print("Clinical data preview:")
    print(preview_df(clinical_df))
    
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False
    clinical_df = pd.DataFrame()  # Empty DataFrame if clinical data is unavailable

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if not clinical_df.empty and not normalized_gene_data.empty:
        # Convert clinical_df to have samples as rows if it doesn't already
        if clinical_df.shape[0] <= 5:  # Small number of rows suggests features as rows
            clinical_df = clinical_df.T
            # Ensure the index is not the trait and covariates
            if clinical_df.index.name is None:
                clinical_df = clinical_df.reset_index()
                
        linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1, join='inner')
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 4. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # 5. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note="Dataset contains clear cell RCC patient expression data with survival information."
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 2592 unique gene symbols
Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE40912.csv

Loading clinical data...
Loaded clinical data with shape: (3, 16)
Clinical data preview:
{'Kidney_Chromophobe': [1.0, 51.0, 1.0], 'GSM1001': [0.0, 78.0, 0.0], 'GSM1002': [nan, 53.0, nan], 'GSM1003': [nan, 41.0, nan], 'GSM1004': [nan, 39.0, nan], 'GSM1005': [nan, 34.0, nan], 'GSM1006': [nan, 66.0, nan], 'GSM1007': [nan, 75.0, nan], 'GSM1008': [nan, 40.0, nan], 'GSM1009': [nan, 63.0, nan], 'GSM1010': [nan, 35.0, nan], 'GSM1011': [nan, nan, nan], 'GSM1012': [nan, nan, nan], 'GSM1013': [nan, nan, nan], 'GSM1014': [nan, nan, nan], 'GSM1015': [nan, nan, nan]}

Linking clinical and genetic data...
Linked data shape: (0, 2596)
Trait column 'Kidney_Chromophobe' not found in linked data.

Performing final validation...
Abnormality detected in the cohort: GSE40912. Preprocessing failed.
Dataset not usable for Kidney_